In [26]:
import random
import calendar
import time
import math
import numpy as np

origintime=0
max_policies=10
max_policy_changes=3
#max_policy_length=365*24*3600
max_policy_length=3600
policy_vars=[3,8]
#policy_m_init=11
#policy_sd_init=4.2
policy_m_init=4
policy_sd_init=1.5


baseclaimrate=1.2
claim_rates=[0.1,0.2]
claim_report_m=11
claim_report_sd=3
claim_payment_rate=1.3

payment_m=9
payment_sd=4
payment_time_delay_m=3
payment_time_delay_sd=1.5


max_claims=1
max_periods=3
max_class=3
max_customer_changes=2

#All timelines are calendar
#Sendingtimes are random, but after origintime

class Sample:
    def __init__(self,polnr):
        self.policies(polnr)
        self.customers()
        self.claims()
        self.combine()
        
    def customers(self):
        times=[p["policystarttime"] for p in self.policy]
        self.customer=[]
        self.customer.append({"customer":"cust"+str(self.policy[0]["policy"]),"customertime":self.policy[0]["policystarttime"],"address":"Here"})
        r=random.sample(times,1)[0]
        if r!=times[0]:
             self.customer.append({"customer":"cust"+str(self.policy[0]["policy"]),"customertime":r,"address":"There"})
        
    def policies(self,polnr):
        #Only one policy is made
        self.policy=[]
        policynr=random.randint(1,max_policies)
        if polnr>0:
            policynr=polnr
        lasttime=origintime+random.lognormvariate(policy_m_init,policy_sd_init)
        for _ in range(random.randint(1,max_policy_changes)):
            newtime=random.uniform(lasttime,lasttime+max_policy_length)
            newpolicy={"policy":policynr,"policystarttime":lasttime,"policyendtime":newtime}
            lasttime=newtime
            ci=0
            for i in policy_vars:
                newpolicy[str("pvar"+str(ci))]=random.sample(set(range(i)),1)[0]
                ci=ci+1
            self.policy.append(newpolicy)

    def claims(self):
        self.claim=[]
        self.claimpayment=[]
        times=[p["policystarttime"] for p in self.policy]
        claimpayment=[]
        claimcount=0
        for ci in range(len(self.policy)):
            rate=baseclaimrate
            for i in range(len(policy_vars)):
                rate=rate+claim_rates[i]*self.policy[ci][str("pvar"+str(i))]
            mu=rate*(float(self.policy[ci]["policyendtime"])-float(self.policy[ci]["policystarttime"]))
            mu=mu/max_policy_length
            claimcounts=math.floor(random.expovariate(1/mu))
            lastclaimtime=self.policy[ci]["policystarttime"]
            for c in range(claimcounts):
                claimtime=random.uniform(lastclaimtime,self.policy[ci]["policyendtime"])
                claimreporttime=claimtime+random.lognormvariate(claim_report_m,claim_report_sd)
                lastclaimtime=claimtime
                claimnumber=str(self.policy[0]["policy"])+"_"+str(claimcount)
                self.claim.append({"claimnumber":claimnumber,"claimtime":claimtime,"claimreporttime":claimreporttime,"claimcounter":claimcount})
                claimcount=claimcount+1
                mupayments=claim_payment_rate
                countpayments=math.floor(random.expovariate(1/mupayments))
                #print("countpayments is "+str(countpayments))
                lastpaytime=self.claim[c]["claimreporttime"]
                for pay in range(countpayments):
                    amount=random.lognormvariate(payment_m,payment_sd)
                    paytime=lastpaytime+random.lognormvariate(payment_time_delay_m,payment_time_delay_sd)
                    lastpaytime=paytime
                    self.claimpayment.append({"claimnumber":self.claim[c]["claimnumber"],"claimcounter":self.claim[c]["claimcounter"],"paytime":paytime,"payment":amount})

    def combine(self):
        self.data={"customer":self.customer,"policy":self.policy,"claim":self.claim,"payment":self.claimpayment}

    def reportdata(self,a):
        #create random delays for all stuff, then combine
        if isinstance(a,dict):
            for k in a.keys():
                print(k)

    def recordsinint(self,fra,til):
        newcustomer=[c for c in self.customer if c["customertime"]<til and c["customertime"]>=fra]
        newpolicy=[c for c in self.policy if c["policystarttime"]<til and c["policystarttime"]>=fra]
        newclaim=[c for c in self.claim if c["claimreporttime"]<til and c["claimreporttime"]>=fra]
        newclaimpayment=[c for c in self.claimpayment if c["paytime"]<til and c["paytime"]>=fra]
        self.newdata={"customer":newcustomer,"policy":newpolicy,"claim":newclaim,"payment":newclaimpayment}
        
    def print(self):
        #print(self.customer)
        #print("\n")
        #print(self.policy)
        #print("\n")
        #print(self.claim)
        #print("\n")
        #print(self.claimpayment)
        print(self.data)
        



In [ ]:
!pip install kafka-python

In [27]:
s=Sample(0)
data_customer=s.customer
data_policy=s.policy
data_claim=s.claim
data_claimpayment=s.claimpayment

for i in range(10):
    s=Sample(i+1)
    data_customer.extend(s.customer)
    data_policy.extend(s.policy)
    data_claim.extend(s.claim)
    data_claimpayment.extend(s.claimpayment)
    

In [ ]:
data=[Sample(i) for i in range(100)]
data_customer=[d.customer for d in data]
data_policy=[d.policy for d in data]
data_claim=[d.claim for d in data]
data_claimpayment=[d.claimpayment for d in data]



In [33]:
for d in data_customer:
    print(d)

{'address': 'Here', 'customer': 'cust2', 'customertime': 209.3058618031297}
{'address': 'There', 'customer': 'cust2', 'customertime': 1675.1796612706603}
{'address': 'Here', 'customer': 'cust1', 'customertime': 39.91089678039432}
{'address': 'There', 'customer': 'cust1', 'customertime': 2619.3868565191538}
{'address': 'Here', 'customer': 'cust2', 'customertime': 83.59820044218392}
{'address': 'Here', 'customer': 'cust3', 'customertime': 30.494245190355837}
{'address': 'There', 'customer': 'cust3', 'customertime': 840.4890973589163}
{'address': 'Here', 'customer': 'cust4', 'customertime': 806.4371306778477}
{'address': 'Here', 'customer': 'cust5', 'customertime': 149.79399605119997}
{'address': 'There', 'customer': 'cust5', 'customertime': 1851.924735134434}
{'address': 'Here', 'customer': 'cust6', 'customertime': 231.61137847840644}
{'address': 'There', 'customer': 'cust6', 'customertime': 3777.033638389481}
{'address': 'Here', 'customer': 'cust7', 'customertime': 483.79650918011544}
{

In [34]:
from kafka import KafkaProducer
from kafka.errors import KafkaError
import json



In [36]:
producer = KafkaProducer(value_serializer=lambda m: json.dumps(m).encode('ascii'),bootstrap_servers=['10.101.12.4:9092'])

for d in data_customer:
    producer.send('customer', d)

for d in data_policy:
    producer.send('policy', d)

for d in data_claim:
    producer.send('claim', d)

for d in data_claimpayment:
    producer.send('claimpayment', d)

producer.flush()

In [35]:
from kafka import KafkaConsumer
consumer = KafkaConsumer('pertest',bootstrap_servers=['10.101.12.4:9092'],auto_offset_reset='earliest')
consumer.subscribe(['pertest'])
for message in consumer:
    print (message)



ConsumerRecord(topic='pertest', partition=0, offset=0, timestamp=1507844981560, timestamp_type=0, key=None, value=b'{"key": "value"}', checksum=-580452069, serialized_key_size=-1, serialized_value_size=16)
ConsumerRecord(topic='pertest', partition=0, offset=1, timestamp=1507845186816, timestamp_type=0, key=None, value=b'{"key": "value"}', checksum=449493981, serialized_key_size=-1, serialized_value_size=16)
ConsumerRecord(topic='pertest', partition=0, offset=2, timestamp=1507845190698, timestamp_type=0, key=None, value=b'{"key": "value"}', checksum=1539751172, serialized_key_size=-1, serialized_value_size=16)
ConsumerRecord(topic='pertest', partition=0, offset=3, timestamp=1507845869436, timestamp_type=0, key=None, value=b'{"address": "Here", "customer": "cust2", "customertime": 209.3058618031297}', checksum=148163309, serialized_key_size=-1, serialized_value_size=75)
ConsumerRecord(topic='pertest', partition=0, offset=4, timestamp=1507845869437, timestamp_type=0, key=None, value=b'{"a

KeyboardInterrupt: 